In [50]:
import fitz
import os
import requests
from zipfile import ZipFile
from io import BytesIO
import sys

In [51]:
# !pip install PyMuPDF

In [52]:
pdf_path = "data\\Datos_abiertos_historicos_etv_2022.pdf"

In [53]:
def extract_links(pdf_path):
    links = []
    doc = fitz.open(pdf_path)    
    page = doc[0]
    for link in page.get_links():
        links.append(
            link['uri']
        )
    doc.close()
    return links

In [54]:
def descompresor(links):
    carpeta = 'data\csv-files'
    success = 0
    fail_download = 0
    fail_process = 0
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    ruta_bitacora = 'data\\bitacora.txt'
    with open(ruta_bitacora, 'w') as log_file:

        #Redirigir la salida estándar al archivo
        sys.stdout = log_file
        for link in links:
            try:
                respuesta = requests.get(link)
                if respuesta.status_code == 200:
                    with ZipFile(BytesIO(respuesta.content)) as archivo_zip:
                        archivo_zip.extractall(carpeta)
                        print(f'Archivo descomprimido desde {link} en {carpeta}')
                        success+=1
                else:
                    print(f'Error al descargar {link}. Código de estado: {respuesta.status_code}')
                    fail_download+=1

            except Exception as e:
                print(f'Error al procesar {link}: {str(e)}')
                fail_process+=1
        print(f"Hubo {success} exitos, {fail_download} fallas al descargar, {fail_process} fallas al descomprimir")
        
        #Restaurar la salida estándar al archivo
        sys.stdout = sys.__stdout__

        print(f"Hubo {success} exitos, {fail_download} fallas al descargar, {fail_process} fallas al descomprimir")

    print(f'Los resultados se han guardado en {ruta_bitacora}')

In [55]:
enlaces = extract_links(pdf_path)

descompresor(enlaces)